重写代码，通过CUDA C代码将gpuarray对象的每个元素都乘2，然后使用pycuda库中的ElementwiseKernel函数来生成代码。

In [11]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray
from time import time
from pycuda.elementwise import ElementwiseKernel

host_data = np.float32( np.random.random(50000000) )

gpu_2x_ker = ElementwiseKernel(
"float *in, float *out", # 设置输入和输出变量
"out[i] = 2*in[i];", # 定义逐个元素的运算
"gpu_2x_ker") # 起一个CUDA C内核函数名称

def speedcomparison():
    t1 = time()
    host_data_2x =  host_data * np.float32(2)
    t2 = time()
    print('total time to compute on CPU: %f' % (t2 - t1))
    device_data = gpuarray.to_gpu(host_data)
    # allocate memory for output
    device_data_2x = gpuarray.empty_like(device_data)
    t1 = time()
    gpu_2x_ker(device_data, device_data_2x)
    t2 = time()
    from_device = device_data_2x.get()
    print('total time to compute on GPU: %f' % (t2 - t1))
    print('Is the host computation the same as the GPU computation? : {}'.format(np.allclose(from_device, host_data_2x) ))
    

if __name__ == '__main__':
    speedcomparison()

total time to compute on CPU: 0.067535
total time to compute on GPU: 0.113093
Is the host computation the same as the GPU computation? : True
